In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True
rdkit.__version__

In [ ]:
smiles = "CCO"

molecule = Chem.MolFromSmiles(smiles)
molecule

In [ ]:
# First, make sure you have the RDKit library installed.
# You can install it using pip: `pip install rdkit-pypi`

from rdkit import Chem
from rdkit.Chem import Draw

# Define your molecule's SMILES string
smiles = 'Cc1ccccc1'  # Example SMILES for toluene

# Create an RDKit molecule object from the SMILES
mol = Chem.MolFromSmiles(smiles)

# Check if the molecule was successfully created
if mol is not None:
    # Generate a 2D depiction of the molecule
    img = Draw.MolToImage(mol)

    # Display the image (you can save it to a file if needed)
    img.show()
else:
    print("Error: Unable to create molecule from SMILES.")


In [ ]:
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import Draw

def get_smiles_from_name(common_name):
    try:
        compounds = pcp.get_compounds(common_name, 'name')
        return compounds[0].isomeric_smiles
    except IndexError:
        return None

def is_valid_molecule(molecule_name):
    smiles = get_smiles_from_name(molecule_name) if not Chem.MolFromSmiles(molecule_name) else molecule_name
    molecule = Chem.MolFromSmiles(smiles)
    return molecule is not None

def visualize_molecule(molecule_name):
    smiles = get_smiles_from_name(molecule_name) if not Chem.MolFromSmiles(molecule_name) else molecule_name
    molecule = Chem.MolFromSmiles(smiles)
    if molecule:
        # Generate a 2D structure for the molecule
        Chem.rdDepictor.Compute2DCoords(molecule)
        # Draw the molecule
        img = Draw.MolToImage(molecule)
        return img
    else:
        print(f"The molecule name '{molecule_name}' is not valid.")
        return None

# Example usage:
common_name = "ethanol"
img = visualize_molecule(common_name)
img.show() if img else None


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from pyscf import gto, symm

def get_point_group_from_smiles(smiles):
    # Create a molecule object from SMILES string
    molecule = Chem.MolFromSmiles(smiles)

    # Add hydrogens
    molecule = Chem.AddHs(molecule)

    # Generate 3D coordinates
    AllChem.EmbedMolecule(molecule)
    AllChem.UFFOptimizeMolecule(molecule)

    # Get atomic coordinates in the format required by PySCF
    conf = molecule.GetConformer()
    atom_coords = ''
    for atom in molecule.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        atom_coords += f"{atom.GetSymbol()} {pos.x} {pos.y} {pos.z};"

    # Define the molecule in PySCF
    mol = gto.M(atom=atom_coords)

    # Detect and print the point group
    mol.build()
    point_group_info = symm.detect_symm(mol)
    point_group = point_group_info[0][0]  # Access the first element of the list
    return point_group

# Example usage:
smiles_string = 'CCO'  # Replace with your molecule's SMILES string
point_group = get_point_group_from_smiles(smiles_string)
print(f'The point group of the molecule is: {point_group}')


In [ ]:
import streamlit as st
import py3Dmol
from stmol import showmol
st.sidebar.title('Show Proteins')
prot_str='1A2C,1BML,1D5M,1D5X,1D5Z,1D6E,1DEE,1E9F,1FC2,1FCC,1G4U,1GZS,1HE1,1HEZ,1HQR,1HXY,1IBX,1JBU,1JWM,1JWS'
prot_list=prot_str.split(',')
bcolor = st.sidebar.color_picker('Pick A Color', '#00f900')
protein=st.sidebar.selectbox('select protein',prot_list)
style = st.sidebar.selectbox('style',['line','cross','stick','sphere','cartoon','clicksphere'])
spin = st.sidebar.checkbox('Spin', value = False)
xyzview = py3Dmol.view(query='pdb:'+protein)
xyzview.setStyle({style:{'color':'spectrum'}})
xyzview.setBackgroundColor(bcolor)
if spin:
    xyzview.spin(True)
else:
    xyzview.spin(False)
xyzview.zoomTo()
showmol(xyzview,height=500,width=800)

In [ ]:
import streamlit as st
from stmol import showmol
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem

st.title('RDKit + Py3DMOL 😀')

def check_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return mol
        else:
            return "Invalid SMILES"
    except:
        return "Invalid SMILES"

def makeblock(smi):
    mol = Chem.MolFromSmiles(smi)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    mblock = Chem.MolToMolBlock(mol)
    return mblock

def render_mol(xyz):
    xyzview = py3Dmol.view()#(width=400,height=400)
    xyzview.addModel(xyz,'mol')
    xyzview.setStyle({'stick':{}})
    xyzview.setBackgroundColor('white')
    xyzview.zoomTo()
    showmol(xyzview,height=500,width=500)

compound_smiles=st.text_input('SMILES please','CC')
check_smiles(smiles=compound_smiles)

blk=makeblock(compound_smiles)
render_mol(blk)

In [55]:
import pointgroup

# Define the coordinates of the atoms in the molecule
# For H2O, the structure can be defined as follows:
atoms = PointGroup(positions= [
    [0.0, 0.0, 0.0],
    [0.758, 0.586, 0.0],
    [-0.758, 0.586, 0.0]],
    symbols = ["O","H","H"])

# Get the point group
pg = atoms.get_point_group()
print(f"The point group of H2O is: {pg}")


The point group of H2O is: C2v


In [65]:
from pointgroup import PointGroup


pg = PointGroup(positions= coordinates_list, 
                symbols=atom_symbols)

print('Point group: ', pg.get_point_group())

Point group:  Cinfv


In [49]:
"SAME AS NEXT CELL, BUT THE OTHER ONE IS SLIGHTLY BETTER"
from rdkit import Chem
from rdkit.Chem import AllChem

# Your SMILES string
smiles = "O"

# Convert SMILES to a molecule object
mol = Chem.MolFromSmiles(smiles)

# Add hydrogens to the molecule
mol_with_h = Chem.AddHs(mol)

# Generate 3D coordinates
AllChem.EmbedMolecule(mol_with_h, AllChem.ETKDG())

x_coord = []
y_coord = []
z_coord = []
# Print the coordinates of each atom
for atom in mol_with_h.GetAtoms():
    pos = mol_with_h.GetConformer().GetAtomPosition(atom.GetIdx())
    print(f"Atom: {atom.GetSymbol()}, Coordinates: {pos.x}, {pos.y}, {pos.z}")
#    x_coord.append(pos.x)
#    y_coord.append(pos.y)
#    z_coord.append(pos.z)


#for item1, item2, item3 in  zip(x_coord, y_coord, z_coord):
#    print(item1)

Atom: O, Coordinates: -0.004602157925855892, 0.3744516469979194, -0.0
Atom: H, Coordinates: 0.8149012384898123, -0.1856569487017614, 0.0
Atom: H, Coordinates: -0.8102990805639566, -0.18879469829615808, -0.0


In [70]:
from rdkit import Chem
from rdkit.Chem import AllChem
from pointgroup import PointGroup

def point_group_from_smiles():  
    smiles = input("Input SMILES : ")

    # Convert SMILES to a molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print("Invalid SMILES input. Please provide a valid SMILES string.")
    else:
        # Add hydrogens to the molecule
        mol_with_h = Chem.AddHs(mol)

        # Generate 3D coordinates
        AllChem.EmbedMolecule(mol_with_h, AllChem.ETKDG())

        # Create a list to store coordinates
        coordinates_list = []
        atom_symbols = []

        # Iterate over atoms and get their coordinates
        for atom in mol_with_h.GetAtoms():
            pos = mol_with_h.GetConformer().GetAtomPosition(atom.GetIdx())
            coordinates_list.append([pos.x, pos.y, pos.z])
            atom_symbols.append(atom.GetSymbol())

    pg = PointGroup(positions= coordinates_list, 
                    symbols=atom_symbols)

    return 'Point group: ', pg.get_point_group()


In [71]:
point_group_from_smiles()

('Point group: ', 'C1')